In [2]:
import snowflake
import getpass
import pprint
from snowflake.connector import DictCursor
from preprocessing import *
from postings_ner import *
import re
import spacy

CONNECTION_PARAMS = {
    "user":None,
    "password":None,
    "account":'snagajob',
    "authenticator":"https://snagajob.okta.com/",
    "database":"PROD_SAJ_SHARE",
    "warehouse":"PROD_WH",
    "schema":"CUSTOMER"
}

In [2]:
CONNECTION_PARAMS["user"] = input("your snowflake user: ")

your snowflake user: nazih.bissat@snagajob.com


In [3]:
CONNECTION_PARAMS["password"] = getpass.getpass("your snowflake password: ")

your snowflake password: ········


In [4]:
query = '''
select QUERYSTRING
from EVENT.EVENTS where EVENTTYPE='Search' and Placement = 'Keyword Search' and DIMDATEKEY >= 20180625
 and QUERYSTRING is not NULL and ((REGEXP_COUNT(QUERYSTRING, '(i=)', 1) = 1) 
 or (REGEXP_COUNT(QUERYSTRING, '(q=)', 1) = 1))
limit 1000;
'''

In [48]:
with snowflake.connector.connect(** CONNECTION_PARAMS) as ctx:
    with ctx.cursor(DictCursor) as cs:
        with cs.execute(query) as results:
            results = [r for r in results]

In [59]:
spacy_entity_types = {'ORG': 'Organization', 'PRODUCT': 'Product', 'CARDINAL': 'Cardinal', 'PERSON': 'Person',
                     'TIME': 'Time', 'DATE': 'Date', 'PERCENT': 'Percent', 'GPE': 'Location_Geo-Political_Area',
                     'WORK_OF_ART': 'Job_Title_or_Degree', 'LANGUAGE': 'Language', 'QUANTITY': 'Quantity',
                     'EVENT': 'Event', 'MONEY': 'Money', 'NORP': 'Nationality_or_Religion_or_Political_group',
                     'ORDINAL': 'Ordinal', 'FAC': 'Facility', 'LOC': 'Location', 'LAW': 'Law'}

stfd_entity_types = {'ORGANIZATION': 'Organization', 'TITLE': 'Title', 'LOCATION': 'Location', 'EMAIL': 'Email',
               'URL': 'URL', 'CITY': 'Location_City', 'STATE_OR_PROVINCE': 'Location_State_or_Province',
               'COUNTRY': 'Location_Country', 'NATIONALITY': 'Nationality', 'RELIGION': 'Religion', 'TITLE': 'Title',
               'IDEOLOGY': 'Ideology', 'CRIMINAL_CHARGE': 'Criminal_Charge', 'CAUSE_OF_DEATH': 'Cause_of_Death',
               'PERSON': 'Person', 'MONEY': 'Money', 'TIME': 'Time', 'ORDINAL': 'Ordinal', 'SET': 'Set',
               'DURATION': 'Duration', 'NUMBER': 'Number', 'PERCENT': 'Percent', 'MISC': 'Miscellaneous',
                'DATE': 'Date'}

In [4]:
def snowflake_to_list(query_results):
    arr = list()
    for r in query_results:
        split = r['QUERYSTRING'].split('|')
        split[:] = [x for x in split if not (x.startswith('w') or x.startswith('l') or x.startswith('s')
                                             or x.startswith('o'))]
        query = re.sub('[a-z]=', '', ', '.join(split))
        arr.append("I am looking for jobs like " + query)
    return arr


results = snowflake_to_list(results)

NameError: name 'results' is not defined

In [62]:
import re
import spacy

nlp = spacy.load('en_core_web_sm')

queries = '.\n\n'.join(results)

with open('/Users/nazih.bissat/Desktop/brat-v1.3_Crunchy_Frog/data/Query_NER/spaCy/queries.txt', 'w') as text_file:
    text_file.write(queries)
    text_file.close()
    
entity_index = 1
entity_ann_file_text = ''
for ent in nlp(queries).ents:
    entity_ann_file_text += 'T' + str(entity_index) + '\t' + spacy_entity_types[ent.label_] + ' ' \
                                + str(ent.start_char) + ' ' + str(ent.end_char) + '\t' \
                                + ent.text + '\n'
    entity_index += 1
    
ann_file = open('/Users/nazih.bissat/Desktop/brat-v1.3_Crunchy_Frog/data/Query_NER/spaCy/queries.ann', 'w')
ann_file.write(entity_ann_file_text)
ann_file.close()

In [3]:
# StanfordNLP NER + RegexNER
startup_corenlp_server()

# CREATING QUERY TEXT FILES AND RESPECTIVE ENTITY ANNOTATION FILES FOR BRAT TEXT ANNOTATION TOOL (StanfordNLP)

with open('/Users/nazih.bissat/Desktop/brat-v1.3_Crunchy_Frog/data/Query_NER/StanfordNLP/queries.txt', 'w') as text_file:
    text_file.write(queries.strip())
    text_file.close()
    
entity_index = 1
entity_ann_file_text = ''
posting_details = annotate_posting(queries)
for s in posting_details['sentences']:
    for e in s['entitymentions']:
        entity_ann_file_text += 'T' + str(entity_index) + '\t' + stfd_entity_types[e['ner']] + ' ' \
                                + str(e['characterOffsetBegin']) + ' ' + str(e['characterOffsetEnd']) + '\t' \
                                + e['text'] + '\n'
        entity_index += 1
        
ann_file = open('/Users/nazih.bissat/Desktop/brat-v1.3_Crunchy_Frog/data/Query_NER/StanfordNLP/queries.ann', 'w')
ann_file.write(entity_ann_file_text)
ann_file.close()
    
shutdown_corenlp_server()

NameError: name 'queries' is not defined

In [66]:
# CREATING POSTING TEXT FILES AND RESPECTIVE ENTITY ANNOTATION FILES FOR BRAT TEXT ANNOTATION TOOL (COGCOMP)
# TRIED BOTH CONLL NER AND ONTONOTES NER (doc.get_ner_conll), ONTONOTES SEEMED TO DETECT MORE

from ccg_nlpy import remote_pipeline

pipeline = remote_pipeline.RemotePipeline()

def find_substring(substring, string):
    """ 
    Returns list of indices where substring begins in string

    >>> find_substring('me', "The cat says meow, meow")
    [13, 19]
    """
    indices = []
    index = -1  # Begin at -1 so index + 1 is 0
    while True:
        # Find next index of substring, by starting search from index + 1
        index = string.find(substring, index + 1)
        if index == -1:  
            break  # All occurrences have been found
        indices.append(index)
    return indices

entity_index = 1
entity_ann_file_text = ''
reg = re.compile(r"^\s+", re.MULTILINE)
test = reg.sub(" ", re.sub(r'  ', ' ', queries))
test = re.sub("\n", "", test)
test = re.sub('(?<=\w)([:+\',/\%])', r' \1', test)
document = pipeline.doc(test)

query_text = document.get_text

with open('/Users/nazih.bissat/Desktop/brat-v1.3_Crunchy_Frog/data/Query_NER/CogComp/queries.txt', 'w') as text_file:
    text_file.write(query_text)
    text_file.close()

entities = dict()
if (str(document.get_ner_ontonotes) != 'NER_ONTONOTES view: this view does not have constituents in your input text. '):
    for entity in document.get_ner_ontonotes:
        mentions = find_substring(entity['tokens'], posting_text)
#           if mentions == []:
#               print(posting_fname, entity['tokens'])
        
        if entity['tokens'] in entities.keys():
            if entities[entity['tokens']] < len(mentions):
                start = mentions[entities[entity['tokens']]]
                entities[entity['tokens']] += 1
                print(start)
                entity_ann_file_text += 'T' + str(entity_index) + '\t' + spacy_entity_types[entity['label']] + ' ' \
                                            + str(start) + ' ' + str(start+len(entity['tokens'])) + '\t' \
                                            + entity['tokens'] + '\n'
    
                entity_index += 1
        else:
            start = mentions[0]
            entities[entity['tokens']] = 1
            entity_ann_file_text += 'T' + str(entity_index) + '\t' + spacy_entity_types[entity['label']] + ' ' \
                                            + str(start) + ' ' + str(start+len(entity['tokens'])) + '\t' \
                                            + entity['tokens'] + '\n'
            entity_index += 1
        
ann_file = open('/Users/nazih.bissat/Desktop/brat-v1.3_Crunchy_Frog/data/Query_NER/CogComp/queries.ann', 'w')
ann_file.write(entity_ann_file_text)
ann_file.close()

ERROR:ccg_nlpy.remote_pipeline:Fail to connect to server.


ConnectionError: HTTPConnectionPool(host='austen.cs.illinois.edu', port=8080): Max retries exceeded with url: /annotate (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x1a1540fbe0>: Failed to establish a new connection: [Errno 61] Connection refused',))

In [68]:
# CREATING POSTING TEXT FILES AND RESPECTIVE ENTITY ANNOTATION FILES FOR BRAT TEXT ANNOTATION TOOL (COGCOMP)
# TRIED BOTH CONLL NER AND ONTONOTES NER (doc.get_ner_conll), ONTONOTES SEEMED TO DETECT MORE

from ccg_nlpy import remote_pipeline

pipeline = remote_pipeline.RemotePipeline()

document = pipeline.doc('This is a test.')

ERROR:ccg_nlpy.remote_pipeline:Fail to connect to server.


ConnectionError: HTTPConnectionPool(host='austen.cs.illinois.edu', port=8080): Max retries exceeded with url: /annotate (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x1a154262b0>: Failed to establish a new connection: [Errno 61] Connection refused',))

In [14]:
# CREATING POSTING TEXT FILES AND RESPECTIVE ENTITY ANNOTATION FILES FOR BRAT TEXT ANNOTATION TOOL (GCP)
# SET ENVIRONMENT VARIABLE 'GOOGLE_APPLICATION_CREDENTIALS=/Users/nazih.bissat/Downloads/My_First_Project-12919f080214.json'
# BEFORE RUNNING JUPYTER NOTEBOOK
from google.cloud import language
import pickle
import re

client = language.LanguageServiceClient()

# Load results pickle file
pkl_file = open('queries-7-3.pkl', 'rb')
queries = pickle.load(pkl_file)
pkl_file.close()

# entity types from enums.Entity.Type
gcp_entity_types = ['UNKNOWN', 'PERSON', 'LOCATION', 'ORGANIZATION','EVENT', 'WORK_OF_ART', 'CONSUMER_GOOD','OTHER']
gcp_entity_dict = {'UNKNOWN': 'Unknown', 'PERSON': 'Person', 'LOCATION': 'Location', 'ORGANIZATION': 'Organization',
                  'EVENT': 'Event', 'WORK_OF_ART': 'Work_of_Art', 'CONSUMER_GOOD': 'Consumer_Good', 'OTHER': 'Other'}

gcp_queries = re.sub(r'\'', "", queries)

document = language.types.Document(
    content=gcp_queries,
    type=language.enums.Document.Type.PLAIN_TEXT,
)
    
with open('/Users/nazih.bissat/Desktop/brat-v1.3_Crunchy_Frog/data/Query_NER/GCP/queries.txt', 'w') as text_file:
    text_file.write(gcp_queries)
    text_file.close()
    
response = client.analyze_entities(
    document=document,
    encoding_type='UTF32',
)
    
entity_index = 1
entity_ann_file_text = ''
dups = dict()
    
for entity in response.entities:
    for mention in entity.mentions:
        start = mention.text.begin_offset
        entity_ann_file_text += 'T' + str(entity_index) + '\t' + gcp_entity_dict[gcp_entity_types[entity.type]] + ' ' \
                                    + str(start) + ' ' + str(start + len(entity.name)) + '\t' \
                                    + entity.name + '\n'
        entity_index += 1
    
ann_file = open('/Users/nazih.bissat/Desktop/brat-v1.3_Crunchy_Frog/data/Query_NER/GCP/queries.ann', 'w')
ann_file.write(entity_ann_file_text)
ann_file.close()

In [70]:
import pickle

output = open('queries-7-3.pkl', 'wb')
pickle.dump(queries, output)
output.close()

In [12]:
text = "I am looking for jobs like mcdonalds."
document = language.types.Document(
    content=text,
    type=language.enums.Document.Type.PLAIN_TEXT,
)
print(document)

type: PLAIN_TEXT
content: "I am looking for jobs like mcdonalds."



In [11]:
import re
text = "I am looking for jobs like mcdonald's."
print(re.sub(r'\'', "", text))

I am looking for jobs like mcdonalds.


In [ ]:
# CREATING POSTING TEXT FILES AND RESPECTIVE ENTITY ANNOTATION FILES FOR BRAT TEXT ANNOTATION TOOL (APACHE OPENNLP)
from subprocess import Popen, PIPE

apache_entity_types = {'person':'Person', 'location':'Location', 'date':'Date', 'time':'Time', 'money':'Money',
                      'organization': 'Organization', 'percentage':'Percent'}

def find_substring(substring, string):
    """ 
    Returns list of indices where substring begins in string

    >>> find_substring('me', "The cat says meow, meow")
    [13, 19]
    """
    indices = []
    index = -1  # Begin at -1 so index + 1 is 0
    while True:
        # Find next index of substring, by starting search from index + 1
        index = string.find(substring, index + 1)
        if index == -1:  
            break  # All occurrences have been found
        indices.append(index)
    return indices

# THIS IS THE COMMAND I RAN FOR IT TO WORK:
# opennlp TokenNameFinder en-ner-person.bin en-ner-location.bin 
# en-ner-time.bin en-ner-money.bin ener-organization.bin en-ner-percentage.bin en-ner-date.bin

batch_posting = ''
for r in results:
    batch_posting += r['JD_SCRUBBED'] + '\n\n\n'
    
with open('//Users/nazih.bissat/Desktop/apache_opennlp/apache-opennlp-1.8.4/bin/batch_postings.txt', 'w') as text_file:
    text_file.write(batch_posting)
    text_file.close()

commands = '''
cd /Users/nazih.bissat/Desktop/apache_opennlp/apache-opennlp-1.8.4/bin
cat batch_postings.txt | opennlp TokenNameFinder en-ner-person.bin en-ner-location.bin en-ner-time.bin en-ner-money.bin en-ner-organization.bin en-ner-percentage.bin en-ner-date.bin
'''

process = Popen('/bin/bash', stdin=PIPE, stdout=PIPE)
out, err = process.communicate(commands.encode('utf-8'))

full_entities = out.decode('utf-8')

posting_index = 1
for posting in full_entities.split('\n\n\n'):
    posting_fname = 'posting' + str(posting_index)
    
    posting_text = re.sub(r'\<START:[a-z]*\> ', '', posting)
    posting_text = re.sub(r' \<END\>', '', posting_text)
    
    with open('/Users/nazih.bissat/Desktop/brat-v1.3_Crunchy_Frog/data/NER/Apache/' + posting_fname + '.txt', 'w') as text_file:
        text_file.write(posting_text)
        text_file.close()
    
    entity_ann_file_text = ''
    counter = 0
    entity_index = 1
    for start in find_substring('<START:', posting):
        end_type = find_substring('>', posting)[2 * (entity_index - 1)]
        e_type = posting[(start+7):(end_type)]
        end_name = find_substring('<END>', posting)[entity_index - 1]
        name = posting[(end_type+2):(end_name-1)]
        entity_ann_file_text += 'T' + str(entity_index) + '\t' + apache_entity_types[e_type] + ' ' \
                                    + str(start-counter) + ' ' + str(start-counter+len(name)) + '\t' \
                                    + name + '\n'
        counter += 15 + len(e_type)
        entity_index += 1
    
    ann_file = open('/Users/nazih.bissat/Desktop/brat-v1.3_Crunchy_Frog/data/NER/Apache/' + posting_fname + '.ann', 'w')
    ann_file.write(entity_ann_file_text)
    ann_file.close()
    
    posting_index += 1
    
commands = '''
rm /Users/nazih.bissat/Desktop/apache_opennlp/apache-opennlp-1.8.4/bin/batch_postings.txt
'''
process = Popen('/bin/bash', stdin=PIPE, stdout=PIPE)

In [ ]:
text = '''
I am looking for jobs like hotel.

I am looking for jobs like full time lowes.

I am looking for jobs like retail.

I am looking for jobs like wendys.

I am looking for jobs like retail.

I am looking for jobs like full time.

I am looking for jobs like retail.

I am looking for jobs like customer service.

I am looking for jobs like parttime , part time.

I am looking for jobs like transportation.

I am looking for jobs like customer service, full time.

I am looking for jobs like all.

I am looking for jobs like hotel.

I am looking for jobs like teen.

I am looking for jobs like team member .

I am looking for jobs like food restaurant.

I am looking for jobs like car.

I am looking for jobs like full time.

I am looking for jobs like wendys.

I am looking for jobs like judge group.
'''